In [1]:
from dotenv import dotenv_values
import os

config = dotenv_values(".env.secret")  # load sensitive variables
print(config.keys())

for c, v in config.items():
    os.environ[c] = v

odict_keys(['WANDB_API_KEY', 'WANDB_NAME', 'WANDB_NOTES', 'WANDB_NOTEBOOK_NAME'])


In [2]:
import wandb

wandb_key = os.environ["WANDB_API_KEY"]
wandb.login(key=wandb_key)

wandb: Currently logged in as: wilber-quito (deepsat). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
import torch

torch.cuda.is_available()

True

In [4]:
import torch

torch.cuda.device_count()

1

In [5]:
import torch

torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1650'

In [6]:
import os

max(1, os.cpu_count() -1)

11

In [7]:
import os
import wandb
import torch
from lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from src.model import NeuralNetwork
from src.data import HousePricingDataModule

project_name = os.environ["WANDB_NAME"]

for _, batch_size in enumerate([256, 512]):

    for i in range(3):

        wandb_logger = WandbLogger(project=project_name, name=f"batch_size_{batch_size}_run_id_{i}")

        data_module = HousePricingDataModule(batch_size=batch_size)
        data_module.prepare_data()

        in_features = data_module.in_features()
        model = NeuralNetwork(in_features)

        max_epochs = 500

        H = {
            "accelerator": "gpu" if torch.cuda.is_available() else "cpu",
            "in_features": in_features,
            "max_epochs": max_epochs,
            "batch_size": batch_size
        }

        for c, v in H.items():
            wandb_logger.experiment.config[c] = v

        trainer = Trainer(
            accelerator=H["accelerator"],
            max_epochs=H["max_epochs"],
            logger=wandb_logger,
            log_every_n_steps=1)

        trainer.fit(model, datamodule=data_module)
        trainer.test(model, datamodule=data_module)
        preds = trainer.predict(model, datamodule=data_module)

        print(preds[0][:5])

        wandb.finish()

[INFO]: Preprocessing training dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1460, 81)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1460, 74)
[INFO]: Dropping categories types with few ocurrence. Current dataframe shape: (1460, 74)
[INFO]: Few ocurrences removed. Current dataframe shape: (1443, 74)
[INFO]: Dropping columns which contains just one type of category. Current dataframe shape: (1443, 74)
[INFO]: Columns with just one type of categroy dropped. Current dataframe shape: (1443, 73)
[INFO]: Preprocessing predict dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1459, 80)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1459, 73)
[INFO]: Input size: 244


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


[INFO]: Preprocessing training dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1460, 81)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1460, 74)
[INFO]: Dropping categories types with few ocurrence. Current dataframe shape: (1460, 74)
[INFO]: Few ocurrences removed. Current dataframe shape: (1443, 74)
[INFO]: Dropping columns which contains just one type of category. Current dataframe shape: (1443, 74)
[INFO]: Columns with just one type of categroy dropped. Current dataframe shape: (1443, 73)
[INFO]: Preprocessing predict dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1459, 80)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1459, 73)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params
------------------------------------
0 | net  | Sequential | 215 K 
------------------------------------
215 K     Trainable params
0         Non-trainable params
215 K     Total params
0.864     Total estimated model params size (MB)


[INFO]: Setting up fit dataset/s
[INFO]: Train dataset size: 981
[INFO]: Validation dataset size: 173
Sanity Checking: |          | 0/? [00:00<?, ?it/s][INFO]: Validation dataloader size: 1
Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


[INFO]: Setting up test dataset/s
[INFO]: Test dataloader size: 1
Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 119.72it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss              5273616384.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
[INFO]: Preprocessing training dataframe...
[INFO]: Dropping columns with full of NA or Identifiers. Current dataframe shape: (1460, 81)
[INFO]: Dropped columns with full of NA or Identifiers. Current dataframe shape: (1460, 74)
[INFO]: Dropping categories types with few ocurrence. Current dataframe shape: (1460, 74)
[INFO]: Few ocurrences removed. Current dataframe shape: (1443, 74)
[INFO]: Dropping columns wh

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


[INFO]: Setting up predict dataset/s
[INFO]: Predict dataloader size: 3
Predicting DataLoader 0: 100%|██████████| 3/3 [00:00<00:00, 62.28it/s] 
tensor([[182188.2969],
        [182188.2969],
        [182188.2969],
        [182188.2969],
        [182188.2969]])


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
test_loss,▁
train_loss,▇██▆▄▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_loss,███▇▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,500
test_loss,5273616384.0
train_loss,6052269568.0
trainer/global_step,1000
val_loss,6835359232.0
